<a href="https://colab.research.google.com/github/Lorenzomoglia/Notebooks/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COSA E' UN CHATBOT RAG E COME FUNZIONA
### Edizione PTMP
##### (PocaTeoria MoltaPratica)

Un modello di intelligenza artificiale generativa ha una vasta conoscenza generale ereditata dalle fonti che sono state utilizzate per allenarlo.

Se vogliamo "insegnare" ad un modello generativo qualcosa che integri la sua conoscenza generale, possiamo seguire due strade:

- Fine Tuning : creiamo un dataset di Fine Tuning, e ri alleniamo il modello di GenAI su questi dati. Presenta pregi e difetti. Ne parliamo in un altro notebook.

- Retrieve and Generate : forniamo al modello le informazioni complementari direttamente durante l'inferenza. E' il paradigma che affronteremo in questo notebook.

## FASE 1 : "RETRIEVE"

In [ ]:
# importo dipendenze e variabili d'ambiente

import pandas as pd
import numpy as np
from ast import literal_eval
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

True

### Questa è la base di conoscenza di produzione di LIA.

In [ ]:
df = pd.read_csv("data\embeddings\embeddings_100.csv", index_col=0)         # carico il csv e lo rendo un dataframe
df['embeddings'] = df['embeddings'].apply(literal_eval).apply(np.array)     # passo gli embeddings da stringa a array
df                                                                          # visualizzo il dataframe

,text,title,n_tokens,embeddings
0,"In TAS, siamo al centro della rivoluzione del ...",Banking 100,176,"[-0.009273371659219265, -0.004997427109628916,..."
1,"La nostra piattaforma ""Bills Presentment and P...",Bills Presentment And Payments 100,168,"[-0.02100216969847679, -0.007880852557718754, ..."
2,La trasformazione delle filiali bancarie è cru...,Branch Transformation 100,240,"[-0.01851186342537403, -0.022544948384165764, ..."
3,"Nel mondo dei pagamenti, offriamo soluzioni di...",Card Issuing And Processing 100,301,"[-0.005936944857239723, -0.000712500128429383,..."
4,Siamo una Fintech specializzata in soluzioni s...,Chi Siamo 100,201,"[0.012219080701470375, -0.012353061698377132, ..."
5,Indirizzo mail per informazioni sui prodotti T...,Contacts,493,"[-0.0047997706569731236, -0.00310545158572495,..."
6,"In TAS, supportiamo aziende del mondo Corporat...",Corporate 100,218,"[0.011922354809939861, -0.022223806008696556, ..."
7,L'Extended Enterprise è fondamentale nell'era ...,Extended Enterprise 100,280,"[0.0034048997331410646, -0.02020329236984253, ..."
8,Le nostre soluzioni di Financial Network Conne...,Financial Networks Connectivity 100,248,"[-0.014843475073575974, -0.0016309183556586504..."
9,"In TAS, supportiamo Fintech innovative come Pa...",Fintech 100,264,"[0.003948881756514311, -0.0030905017629265785,..."


### Osserviamo come un "embedding" sia effettivamente un array di 1536 floats.

In [ ]:
embs = [emb for emb in df.embeddings]
print("Tipo:", type(embs[0]))
print("Lunghezza:", len(embs[0]))


Tipo: <class 'numpy.ndarray'>
Lunghezza: 1536


### Ora teniamo soltanto le parti che ci servono: colonne title, text e embeddings.

In [ ]:
df = df[["title", "text", "embeddings"]]

In [ ]:
df

,title,text,embeddings
0,Banking 100,"In TAS, siamo al centro della rivoluzione del ...","[-0.009273371659219265, -0.004997427109628916,..."
1,Bills Presentment And Payments 100,"La nostra piattaforma ""Bills Presentment and P...","[-0.02100216969847679, -0.007880852557718754, ..."
2,Branch Transformation 100,La trasformazione delle filiali bancarie è cru...,"[-0.01851186342537403, -0.022544948384165764, ..."
3,Card Issuing And Processing 100,"Nel mondo dei pagamenti, offriamo soluzioni di...","[-0.005936944857239723, -0.000712500128429383,..."
4,Chi Siamo 100,Siamo una Fintech specializzata in soluzioni s...,"[0.012219080701470375, -0.012353061698377132, ..."
5,Contacts,Indirizzo mail per informazioni sui prodotti T...,"[-0.0047997706569731236, -0.00310545158572495,..."
6,Corporate 100,"In TAS, supportiamo aziende del mondo Corporat...","[0.011922354809939861, -0.022223806008696556, ..."
7,Extended Enterprise 100,L'Extended Enterprise è fondamentale nell'era ...,"[0.0034048997331410646, -0.02020329236984253, ..."
8,Financial Networks Connectivity 100,Le nostre soluzioni di Financial Network Conne...,"[-0.014843475073575974, -0.0016309183556586504..."
9,Fintech 100,"In TAS, supportiamo Fintech innovative come Pa...","[0.003948881756514311, -0.0030905017629265785,..."


### Facciamo ora una funzione che, con una chiamata a ada-002, calcola l'embedding di una frase di nostra scelta.

In [ ]:
# Creiamo un'istanza dell'SDK di openai. Lo considereremo un singleton in tutto il notebook.
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def genera_embeddings(testo:str) -> np.ndarray:

    response = client.embeddings.create(input=testo,
                                        model='text-embedding-ada-002')
    return response.data[0].embedding

In [ ]:
nuovi_embeddings = genera_embeddings("Ciao, mi chiamo Lorenzo.")

print(nuovi_embeddings)

[-0.006411836948245764, 0.0025656740181148052, -0.004057490732520819, -0.0391472689807415, -0.03098219633102417, 0.014238785952329636, -0.002271380741149187, -0.014564387500286102, -0.02254161424934864, -0.014777280390262604, 0.040249302983284, 0.01537838950753212, 0.005071863066405058, -0.022904783487319946, 0.020199790596961975, -0.021189117804169655, 0.020187268033623695, -0.01238536462187767, 0.047487664967775345, -0.005488256923854351, 0.008352920413017273, -0.010726051405072212, 0.005879604257643223, -0.0006003269809298217, -0.00359100429341197, -0.015140450559556484, 0.005914042703807354, -0.0066435146145522594, 0.042278047651052475, -0.061613745987415314, 0.02282964624464512, 0.013312075287103653, -0.011846870183944702, 0.0029053636826574802, 0.005654188338667154, -0.027350490912795067, -0.006737438030540943, -0.009611493907868862, 0.0025171469897031784, -0.012617042288184166, 0.029604652896523476, -0.017294427379965782, 0.0054319025948643684, -0.02325543202459812, -0.002067880

### Definiamo ora una funzione per la distanza coseno.
PS: ricordiamo, per i più ruspanti, che la distanza coseno è il prodotto scalare / il prodotto delle norme.

Utilizziamo la distanza coseno perchè la magnitudine non ci interessa. Perchè non ci interessa? Perchè i modelli di embeddings restituiscono sempre array della stessa lunghezza. (nel caso di ada 1536).

Contrariamente a quanto potrebbe sembrare, migliore è il modello, minore è la lunghezza dei vettori che produce. I modelli precedenti ad ada-002 restituivano embeddings con lunghezze comprese tra 10.000 e 20.000 a seconda del modello, rendendo piu lunghe le elaborazioni. Si poteva ridurre la dimensionalità con la PCA, ma si perdeva MOLTA accuratezza.

In [ ]:
def distanza_coseno(a:np.ndarray,b:np.ndarray)->float:

    return 1 - np.dot(a,b)/np.linalg.norm(a)*np.linalg.norm(b)

### Usiamo pandas per creare una colonna "distanza" che contiene la distanza_coseno tra l'embedding della funzione che noi gli diamo e tutte quelle nel dataframe

In [ ]:
b = genera_embeddings("avete una soluzione antifrode?")

# creiamo la colonna distanza
df['distanza'] = df['embeddings'].apply(lambda a : distanza_coseno(a,b))

# ordiniamo il df per quella distanza
df.sort_values('distanza')



,title,text,embeddings,distanza
10,Fraud Management 100,"In un'era digitale in rapida evoluzione, la ge...","[-0.011561820283532143, -0.009850455448031425,...",0.219493
19,Saluti e Formalità,ciao! ciao ceu salve buondì ciaoooo buongiorno...,"[-0.02119011990725994, 0.015528937801718712, -...",0.240475
2,Branch Transformation 100,La trasformazione delle filiali bancarie è cru...,"[-0.01851186342537403, -0.022544948384165764, ...",0.248291
4,Chi Siamo 100,Siamo una Fintech specializzata in soluzioni s...,"[0.012219080701470375, -0.012353061698377132, ...",0.250716
0,Banking 100,"In TAS, siamo al centro della rivoluzione del ...","[-0.009273371659219265, -0.004997427109628916,...",0.254012
6,Corporate 100,"In TAS, supportiamo aziende del mondo Corporat...","[0.011922354809939861, -0.022223806008696556, ...",0.254255
16,Pubblica Amministrazione 100,"In TAS, supportiamo la trasformazione digitale...","[0.004549538251012564, 0.0082161296159029, 0.0...",0.262054
18,Real Time Payments 100,Il Parlamento Europeo ha recentemente approvat...,"[-0.015886729583144188, -0.002333404030650854,...",0.262123
8,Financial Networks Connectivity 100,Le nostre soluzioni di Financial Network Conne...,"[-0.014843475073575974, -0.0016309183556586504...",0.263480
1,Bills Presentment And Payments 100,"La nostra piattaforma ""Bills Presentment and P...","[-0.02100216969847679, -0.007880852557718754, ...",0.266169


### Ora stessa cosa, ma facciamo una funzione, che tiene solo la prima riga, e estrae text e title

In [ ]:
def esito_ricerca(frase:str) -> str:

    # genero embeddings della mia frase, e valuterò le
    b = genera_embeddings(frase)

    # creiamo la colonna distanza
    df['distanza'] = df['embeddings'].apply(lambda a : distanza_coseno(a,b))

    # prendiamo il primo elemento del df ordinato per la distanza
    riga_migliore = df.sort_values('distanza').iloc[0]

    # impacchettiamo il tutto in una stringa con forma "titolo : testo"
    risultato = f"{riga_migliore['title']} : {riga_migliore['text']}"

    return risultato

### Adesso siamo in grado, a partire da una frase, di estrarre il testo più pertinente dalla base di conoscenza e il suo titolo.

In [ ]:
esito_ricerca("dove sono le vostre sedi?")

"Saluti e Formalità : ciao! ciao ceu salve buondì ciaoooo buongiorno buonasera ciao gioia ciao bello ciao bella ciao LIA buon pomeriggio hola good morning bonjour Dovrai rispondere a queste domande con frasi gentili e positive, presentandoti sempre come LIA. Se l'utente dice di avere un problema, allora rimanda a solutions@tasgroup.eu. Il numero di telefono dell'assistenza clienti è 051 458011. Se l'utente vuole parlare con un commerciale, allora rimanda a solutions@tasgroup.eu. Se l'utente vuole mettersi in contatto, allora digli che può usare il form di contatto al link https://tasgroup.eu/contacts. Sedi in Italia: - Milano: Via Serviliano Lattuada 25 - Roma: Via Cristoforo Colombo 149 - Bologna: Via del Lavoro 47, Casalecchio di Reno - Parma: Via Licinio Ferretti 5/A - Prato: Via Traversa Pistoiese 83 - Siena: Via A. Marzi 4 Sedi Estere: - Francia: traverse des Brucs 15, Valbonne Sophia Antipolis - Svizzera: Via Serafino Balestra 22B, Lugano - Germania: Wilhelm-Hale-Str 50, München 

## Passiamo ora alla seconda parte: GENERATE

Ora passiamo alla parte vera e propria della generazione della risposta, ma parliamo prima un attimo dei modi in cui si può generare un testo generico utilizzando AI generativa, focalizzandoci non sulla teoria ma sulla pratica.

A prescinere dal modello usato, da chi lo ha realizzato, e dall'infrastruttura sistemistica che si occupa dell'inferenza, i metodi sono progettati per potere essere inferiti in due modi:

- con un PROMPT (modelli con nome xxx-instruct)
- con un CHAT COMPLETION (modelli con nome xxx-chat)

I modelli instruct sono sempre meno utilizzati, ma consentono di generare un testo a partire da un'istruzione in linguaggio naturale. Ecco un esempio:



In [ ]:
response = client.completions.create(prompt="Raccontami una storia divertente in italiano",
                                    model = "gpt-3.5-turbo-instruct",
                                    max_tokens=1000,
                                    temperature=1
                                    )

print(response.choices[0].text)



Una volta in una scuola elementare c'era un bambino di nome Luca che aveva sempre una grande passione per gli animali. Un giorno, durante la pausa, Luca trovò un insetto molto strano nel cortile della scuola. Era verde fluo con una strana forma a spirale e sembrava molto arrabbiato.

Senza pensarci due volte, Luca lo prese in mano e corse verso la classe per mostrarlo ai suoi compagni. Ma, non appena entrò nella stanza, l'insetto lo morse sul dito, facendolo gridare e far cadere l'insetto per terra.

Improvvisamente, tutto il resto della classe sembrò improvvisamente svegliarsi da un sonno profondo e povocare risate generali. "Che cosa è successo?" chiese Luca, ancora un po' confuso.

"Il tuo amico insetto era un insetto dormiglione, che può causare a chiunque lo tocchi di addormentarsi!" spiegò il suo amico Marco tra una risata e l'altra.

Tutti si diedero da fare per cercare di svegliare Luca che, a quel punto, si trovava a dormire sul pavimento insieme alla sua "amico" insetto. Al

I modelli di chat, ovvero quelli più potenti e moderni, seguono una sintassi di generazione del testo che invia una "chat" al sistema, il quale deve generare il completamento più logico a quella chat.

Viene inviata al modello una lista di messaggi, ciascun messaggio è un dizionario con chiavi "role" e "content".

I ruoli possono essere "system", "user" o "assistant" o "function". In questa sede ci serviranno solo i primi due:

- I messaggi "system" contengono le istruzioni che vanno "dette all'orecchio" del modello prima che generi la risposta. Solitamente si utilizza per le istruzioni o per informazioni aggiuntive.

- I messaggi "user" contengono la vera e propria domanda che il modello dovrà inferire, e solitamente sono l'ultimo messaggio.




In [ ]:
messaggio_system = "Sei un algoritmo cantastorie. Parla in napoletano"
messaggio_user = " raccontami una storia divertente"

response = client.chat.completions.create(
    messages = [
        {"role" : "system", "content" : messaggio_system},
        {"role": "user", "content": messaggio_user}
    ],
    model = "gpt-3.5-turbo",
    temperature = 1
)

print(response.choices[0].message.content)


Napule, 'na vota, c'era 'na famiglia ricca ricca che aveva 'nu piccerillo cchiù pazzarello ca na capa 'e pipistrello. Chest'ommo cacchio, nun sapeva do' mué e chist'uocchie 'e coppa: 'e faveva cadere arrecanno rierzo e riso dint'a casa. E tutto era na festa! 'O taglio 'e capelli? Lostregga! 'A scuola? Chiacchiere e risate! E n'ata cosa, quanno era 'a semmenza do'dimane, chest'uocchie gua 'e Patrick 'a stella 'e mare. Pe' 'a festa 'e natale, avevanne magnato en 'o presepe, e chest'ommo pazzarello,- mentre stava 'e aspettà 'a gesù bambino,- rumpette aranto 'o presepe e dicette:" 'O regalo comme l'mmaggio 'a miezz'ate". 'E risate scocciavono dint''à casa, e risucceva chiddu'uaggio 'o bbadalucco! E nisciuno cchiù seppe 'e uocchie 'e Patrid', ca vide e spaccò, cercanno 'o dulore dint'ôrideuceallo.


### Possiamo anche implementare più messaggi system, ad esempio uno con una istruzione e uno con dei dati aggiuntivi

In [ ]:
messaggio_system = "Sei un algoritmo che aiuta gli studenti. Fai ciò che ti viene chiesto"


messaggio_system2 = """
Ludovico Ariosto, uno degli scrittori più influenti del Rinascimento italiano, è noto soprattutto per il suo poema epico "Orlando Furioso". Nato il 8 settembre 1474 a Reggio Emilia, Ariosto crebbe in una famiglia benestante grazie alla posizione di suo padre come comandante della fortezza di Reggio. La famiglia si trasferì poi a Ferrara, dove Ludovico trascorse gran parte della sua vita.

La formazione di Ariosto fu dapprima legale, come desiderato dal padre, ma ben presto si orientò verso gli studi umanistici. Studiò sotto il reggente della Scuola d'Este, Gregorio da Spoleto, che gli insegnò greco e latino e gli trasmise l'amore per la letteratura classica. Durante gli anni universitari, Ariosto iniziò a scrivere poesie, influenzato dai lavori di poeti come Virgilio e Ovidio.

Dopo l'università, Ariosto entrò al servizio della corte degli Este a Ferrara, dove rimase per la maggior parte della sua vita lavorativa. Qui, iniziò la sua carriera come diplomatico e poi come capitano della fortezza di Canossa. Durante questo periodo, si dedicò anche alla scrittura e al teatro, producendo commedie che rispecchiavano lo stile e l'umorismo della commedia classica latina e delle opere di Plauto.

La sua opera più celebre, "Orlando Furioso", fu pubblicata per la prima volta nel 1516. Il poema è un ampliamento del lavoro iniziato da Matteo Maria Boiardo con "Orlando Innamorato". "Orlando Furioso" mescola elementi romantici, avventurosi e fantastici, raccontando le storie di numerosi cavalieri, dame, maghi e mostri, con un intreccio che si snoda attraverso vari continenti e sfide eroiche. La narrativa complessa e l'uso di una lingua ricca e variegata fecero di questo poema un capolavoro del Rinascimento e un modello per la letteratura epica successiva.

Ariosto rivedette "Orlando Furioso" due volte, pubblicando edizioni rinnovate nel 1521 e nel 1532, quest'ultima solo un anno prima della sua morte avvenuta il 6 luglio 1533 a Ferrara. Oltre a essere un epico poeta, Ariosto fu anche un abile amministratore e funzionario, incarichi che gli furono spesso gravosi ma che svolse con dedizione.

La vita di Ludovico Ariosto fu segnata dall'equilibrio tra le sue responsabilità alla corte degli Este e il suo impegno letterario. Nonostante le pressioni e le sfide della vita di corte, riuscì a creare opere che hanno lasciato un segno indelebile nella letteratura italiana e mondiale. La sua abilità nel tessere trame complesse, il suo uso magistrale della lingua e la sua profonda comprensione della natura umana lo rendono una figura di spicco del suo tempo.
"""


messaggio_user = "Fammi un riassunto in 50 parole"    # oppure "in quali anni rivise la sua opera?"


response = client.chat.completions.create(
    messages = [

        {"role" : "system", "content" : messaggio_system},
        {"role" : "system", "content" : messaggio_system2},
        {"role": "user", "content": messaggio_user}
    ],
    model = "gpt-3.5-turbo",
    temperature = 1
)

print(response.choices[0].message.content)

Ludovico Ariosto, poeta rinascimentale italiano, nacque nel 1474 a Reggio Emilia. Autore dell'epico "Orlando Furioso", crebbe a Ferrara, dove lavorò alla corte degli Este. La sua opera, influenzata dalla classicità, narra storie avventurose e romantiche. Ariosto fu anche amministratore e morì nel 1533, lasciando un'impronta duratura.


### Ora uniamo le due parti viste fin'ora.

Uniamo la funzione esito_ricerca che fa la ricerca sul vectorStore e la posizioniamo in modo che popoli il secondo messaggio system (messaggio_system2), facendo la ricerca sul contenuto della domanda dell'utente (messaggio_user).

Modifichiamo anche il messaggio_system1 per ragguagliare il modello sulla tipologia di dati che

In [ ]:
messaggio_system = "Sei un chatbot aziendale che deve rispodnere a delle domande. Ti passerò la documentazione aziendale collegata alle domande"

messaggio_user = "avete un sofrtware antifrode?"    # oppure "in quali anni rivise la sua opera?"

messaggio_system2 = esito_ricerca(messaggio_user)

response = client.chat.completions.create(
    messages = [
        {"role" : "system", "content" : messaggio_system},
        {"role" : "system", "content" : messaggio_system2},
        {"role": "user", "content": messaggio_user}
    ],
    model = "gpt-3.5-turbo",
    temperature = 1
)

print(response.choices[0].message.content)

Sì, offriamo una soluzione avanzata per la gestione delle frodi chiamata "Fraud Management 100". Questa soluzione utilizza tecnologie come Machine Learning e Big Data per fornire protezione proattiva e in tempo reale contro le frodi. I principali componenti della nostra soluzione includono "Fraud Protect", "Sanction Screening", "SCA Exemption" e "3D Secure". Questi strumenti aiutano a monitorare le transazioni sospette, verificare contro liste di sanzioni, ottimizzare l'Autenticazione Forte del Cliente e migliorare la sicurezza delle transazioni e-commerce. Clienti come Poste Italiane, ICCREA, Monte dei Paschi di Siena e Nexi si affidano a noi per una difesa robusta contro le frodi, proteggendo la loro reputazione e fiducia.
